In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2016','era_2016.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 544MB
Dimensions:         (valid_time: 2928, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2016-01-01 ... 2016-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:17 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 543MB
Dimensions:         (valid_time: 2928, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2016-01-01 ... 2016-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:17 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2016-01-01 00:00:00 1000.0         49.25    -124.85    82.287781  0.003168
                                            -124.60    76.617859  0.002930
                                            -124.35    69.000671  0.002679
                                            -124.10    66.033875  0.002634
                                            -123.85    71.324890  0.002739
...                                                          ...       ...
2016-12-31 21:00:00 1000.0         24.50    -68.10     67.924545  0.010523
                                            -67.85     68.699936  0.010595
                                            -67.60     69.666733  0.010692
                                            -67.35     70.655014  0.010790
                                            -67.10     71.514389  0.010867

[67929600 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2016-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:17 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2016-12-31 1000.0         49.25    -124.85    83.404022  0.006377
                                   -124.60    80.790833  0.006375
                                   -124.35    78.369232  0.006279
                                   -124.10    75.888649  0.006194
                                   -123.85    72.811127  0.006189
...                                                 ...       ...
                          24.50    -68.10     78.021904  0.014781
                                   -67.85     78.080910  0.014786
                                   -67.60     78.141609  0.014790
                                   -67.35     78.181313  0.014792
                                   -67.10     78.204353  0.014789

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2016-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 83.4 ... 78.2
    q               (valid_time, latitude, longitude) float32 93kB 0.006377 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:17 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2016_county_shapefile','cb_2016_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,19,107,00465242,0500000US19107,19107,Keokuk,06,1500067253,1929323,"POLYGON ((-92.41199 41.50955, -92.35539 41.509..."
1,19,189,00465283,0500000US19189,19189,Winnebago,06,1037261946,3182052,"POLYGON ((-93.97076 43.49960, -93.88843 43.499..."
2,20,093,00485011,0500000US20093,20093,Kearny,06,2254696689,1133601,"POLYGON ((-101.54192 37.91457, -101.54186 37.9..."
3,20,123,00485026,0500000US20123,20123,Mitchell,06,1817632928,44979981,"POLYGON ((-98.49007 39.24167, -98.49005 39.263..."
4,20,187,00485055,0500000US20187,20187,Stanton,06,1762104518,178555,"POLYGON ((-102.04190 37.54119, -102.04189 37.5..."
...,...,...,...,...,...,...,...,...,...,...
3228,72,123,01804542,0500000US72123,72123,Salinas,13,179660999,115910809,"POLYGON ((-66.34257 17.99366, -66.34053 17.995..."
3229,47,017,01639729,0500000US47017,47017,Carroll,06,1547925982,6308712,"POLYGON ((-88.70600 35.79604, -88.70667 35.798..."
3230,51,183,01690257,0500000US51183,51183,Sussex,06,1269664389,6755317,"POLYGON ((-77.61620 36.87920, -77.58713 36.894..."
3231,55,013,01581066,0500000US55013,55013,Burnett,06,2127856271,151932123,"POLYGON ((-92.88571 45.64602, -92.88442 45.652..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-92.41199 41.50955, -92.35539 41.509...
1       POLYGON ((-93.97076 43.49960, -93.88843 43.499...
2       POLYGON ((-101.54192 37.91457, -101.54186 37.9...
3       POLYGON ((-98.49007 39.24167, -98.49005 39.263...
4       POLYGON ((-102.04190 37.54119, -102.04189 37.5...
                              ...                        
3228    POLYGON ((-66.34257 17.99366, -66.34053 17.995...
3229    POLYGON ((-88.70600 35.79604, -88.70667 35.798...
3230    POLYGON ((-77.61620 36.87920, -77.58713 36.894...
3231    POLYGON ((-92.88571 45.64602, -92.88442 45.652...
3232    POLYGON ((-89.04779 45.98234, -89.03148 45.982...
Name: geometry, Length: 3233, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -92.411995  41.509548
     1   -92.355389  41.509646
     2   -92.345287  41.509677
     3   -92.297494  41.509790
     4   -92.287642  41.509828
...             ...        ...
3232 174 -89.047475  45.796780
     175 -89.047602  45.881586
     176 -89.047601  45.895355
     177 -89.047753  45.924363
     178 -89.047795  45.982345

[1039365 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3232 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 5min 53s
Wall time: 6min


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,69.01187,0.00752
1,74.176746,0.006889
2,51.31893,0.006542
3,61.874895,0.007756
4,48.67611,0.006305
...,...,...
3228,<NA>,<NA>
3229,65.366128,0.009049
3230,66.51921,0.008609
3231,70.727566,0.006043


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
AFFGEOID      object
GEOID         object
NAME          object
LSAD          object
ALAND          int64
AWATER         int64
geometry    geometry
fips           int32
dtype: object

In [15]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Keokuk,"POLYGON ((-92.41199 41.50955, -92.35539 41.509...",19107
1,Winnebago,"POLYGON ((-93.97076 43.49960, -93.88843 43.499...",19189
2,Kearny,"POLYGON ((-101.54192 37.91457, -101.54186 37.9...",20093
3,Mitchell,"POLYGON ((-98.49007 39.24167, -98.49005 39.263...",20123
4,Stanton,"POLYGON ((-102.04190 37.54119, -102.04189 37.5...",20187
...,...,...,...
3228,Salinas,"POLYGON ((-66.34257 17.99366, -66.34053 17.995...",72123
3229,Carroll,"POLYGON ((-88.70600 35.79604, -88.70667 35.798...",47017
3230,Sussex,"POLYGON ((-77.61620 36.87920, -77.58713 36.894...",51183
3231,Burnett,"POLYGON ((-92.88571 45.64602, -92.88442 45.652...",55013


In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Keokuk,"POLYGON ((-92.41199 41.50955, -92.35539 41.509...",19107,69.01187,0.00752
1,Winnebago,"POLYGON ((-93.97076 43.49960, -93.88843 43.499...",19189,74.176746,0.006889
2,Kearny,"POLYGON ((-101.54192 37.91457, -101.54186 37.9...",20093,51.31893,0.006542
3,Mitchell,"POLYGON ((-98.49007 39.24167, -98.49005 39.263...",20123,61.874895,0.007756
4,Stanton,"POLYGON ((-102.04190 37.54119, -102.04189 37.5...",20187,48.67611,0.006305
...,...,...,...,...,...
3228,Salinas,"POLYGON ((-66.34257 17.99366, -66.34053 17.995...",72123,<NA>,<NA>
3229,Carroll,"POLYGON ((-88.70600 35.79604, -88.70667 35.798...",47017,65.366128,0.009049
3230,Sussex,"POLYGON ((-77.61620 36.87920, -77.58713 36.894...",51183,66.51921,0.008609
3231,Burnett,"POLYGON ((-92.88571 45.64602, -92.88442 45.652...",55013,70.727566,0.006043


In [17]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Keokuk,"POLYGON ((-92.41199 41.50955, -92.35539 41.509...",19107,69.01187,0.00752
1,Winnebago,"POLYGON ((-93.97076 43.49960, -93.88843 43.499...",19189,74.176746,0.006889
2,Kearny,"POLYGON ((-101.54192 37.91457, -101.54186 37.9...",20093,51.31893,0.006542
3,Mitchell,"POLYGON ((-98.49007 39.24167, -98.49005 39.263...",20123,61.874895,0.007756
4,Stanton,"POLYGON ((-102.04190 37.54119, -102.04189 37.5...",20187,48.67611,0.006305
...,...,...,...,...,...
3226,Marshall,"POLYGON ((-89.72432 34.99521, -89.70661 34.995...",28093,64.983245,0.009418
3229,Carroll,"POLYGON ((-88.70600 35.79604, -88.70667 35.798...",47017,65.366128,0.009049
3230,Sussex,"POLYGON ((-77.61620 36.87920, -77.58713 36.894...",51183,66.51921,0.008609
3231,Burnett,"POLYGON ((-92.88571 45.64602, -92.88442 45.652...",55013,70.727566,0.006043


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Keokuk,19107,69.011870029231247,0.007520353121707
1,Winnebago,19189,74.176745977597292,0.006888699779731
2,Kearny,20093,51.318930014667124,0.006541614280564
3,Mitchell,20123,61.874895069735160,0.007756109527335
4,Stanton,20187,48.676109830865755,0.006304735541207
...,...,...,...,...
3226,Marshall,28093,64.983245010960246,0.009418266375998
3229,Carroll,47017,65.366127893084510,0.009049150860150
3230,Sussex,51183,66.519209690833520,0.008608610839215
3231,Burnett,55013,70.727566348851937,0.006042677786506


In [20]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
18,Autauga,1001,63.254521318504011,0.009808564686794
19,Baldwin,1003,71.422116274740588,0.011820886003973
769,Barbour,1005,64.754889774262210,0.010124381910884
2967,Bibb,1007,63.356280062205528,0.009619432122968
883,Blount,1009,64.131153759126619,0.009339549844518
...,...,...,...,...
2062,Sweetwater,56037,57.237399581641853,0.003801776653520
1267,Teton,56039,66.660410983126866,0.003749517844549
1851,Uinta,56041,57.308615888470293,0.003686725710527
1861,Washakie,56043,52.376161816483538,0.003961531004313


In [21]:
county_var.to_pickle('era_2016.pkl')